# Hot Sauce NLP


In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib

In [2]:
def fetch_page(page):
    print(f"scraping page {page + 1}")
    response = requests.get(
        f'https://hotsaucefever.com/hotsauce/',
        headers={"Accept-Language":"en-US"}
    )
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [3]:

def add_sauces_to_dict(soup, dict):
    for sauce_html in soup.find_all(class_='detail'):
        dict['Hot Sauce'].append(sauce_html.find('h2').find('a').string)
        dict['Rating'].append(sauce_html.find('h3').find('span').string)
        dict['Makers'].append(
            sauce_html.find(class_="list-txt").get_text())

In [4]:
def create_sauces_df(max_page):
    sauces_dict = {'Hot Sauce': [], 'Rating': [], 'Makers': []}
    for page in range(max_page):
        soup = fetch_page(page)
        add_sauces_to_dict(soup, sauces_dict)
    return pd.DataFrame.from_dict(sauces_dict)

In [5]:
sauces_df = create_sauces_df(1)
sauces_df.shape

scraping page 1


(25, 3)

In [6]:
sauces_df


,Hot Sauce,Rating,Makers
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Maker: Whitehouse Station Sauce Company, White..."
1,Jersey Barnfire - Black Garlic,None,"Maker: Jersey Barnfire, Sparta, NJ, United Sta..."
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Maker: Roofeeo Landis, Los Angeles , CA, Unite..."
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Maker: Ric Orlando's Best, West Hurley, NY, Un..."
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"Maker: Hell's Kitchen Hot Sauce, New York, NY,..."
5,Pickle Monster Hot Sauce - Spicy Boi Blend,None,"Maker: Pickle Monster Hot Sauce, Philadelphia,..."
6,Pickle Monster Hot Sauce - Pickle Monster,None,"Maker: Pickle Monster Hot Sauce, Philadelphia,..."
7,Swamp Dragon - Tequila Hot Sauce,None,"Maker: Swamp Dragon, Baton Rouge, LA, United S..."
8,Cepera - Molho de Pimenta Vermelha: Salsa de A...,None,"Maker: Cepera, Brazil\n \nPepper(s)..."
9,Tacoma Sauce Co. - Nero,N/A,"Maker: Tacoma Sauce Co., Tacoma, United States..."


In [7]:
# new data frame with split value columns
new = sauces_df["Makers"].str.split("Maker:", n =1, expand = True)

# making separate first name column from new data frame
#books_df["Makers"]= new[0]
  
# making separate last name column from new data frame
#books_df["Peppers"]= new[1]

#new1 = new[1].str.split("Pepper(s):", n = 1, expand = True)

In [8]:
new


,0,1
0,,"Whitehouse Station Sauce Company, Whitehouse ..."
1,,"Jersey Barnfire, Sparta, NJ, United States\n ..."
2,,"Roofeeo Landis, Los Angeles , CA, United Stat..."
3,,"Ric Orlando's Best, West Hurley, NY, United S..."
4,,"Hell's Kitchen Hot Sauce, New York, NY, Unite..."
5,,"Pickle Monster Hot Sauce, Philadelphia, PA, U..."
6,,"Pickle Monster Hot Sauce, Philadelphia, PA, U..."
7,,"Swamp Dragon, Baton Rouge, LA, United States\..."
8,,"Cepera, Brazil\n \nPepper(s):\n ..."
9,,"Tacoma Sauce Co., Tacoma, United States\n ..."


In [9]:
new1 = new[1].str.split(":", expand = True)


# 

In [10]:
cols = [0, 1, 2]
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip(' \n%'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\nPepper(s)'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('SHU'))

In [11]:
new1

,0,1,2
0,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None
1,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None
2,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None
3,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None
4,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None
5,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","abanero, Jalapeño, Serrano",None
6,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","Jalapeño, Serrano",None
7,"wamp Dragon, Baton Rouge, LA, United Stat",None,None
8,"Cepera, Brazil\n",Jalapeño,None
9,"Tacoma Sauce Co., Tacoma, United States\n ...",abanero\n \n,"3,000"


In [12]:
result = pd.concat([sauces_df, new1], axis=1, ignore_index=True)

In [13]:
result.drop(result.columns[[2]], axis=1, inplace=True)

In [19]:
result

,0,1,3,4,5
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None
1,Jersey Barnfire - Black Garlic,None,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None
5,Pickle Monster Hot Sauce - Spicy Boi Blend,None,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","abanero, Jalapeño, Serrano",None
6,Pickle Monster Hot Sauce - Pickle Monster,None,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","Jalapeño, Serrano",None
7,Swamp Dragon - Tequila Hot Sauce,None,"wamp Dragon, Baton Rouge, LA, United Stat",None,None
8,Cepera - Molho de Pimenta Vermelha: Salsa de A...,None,"Cepera, Brazil\n",Jalapeño,None
9,Tacoma Sauce Co. - Nero,N/A,"Tacoma Sauce Co., Tacoma, United States\n ...",abanero\n \n,"3,000"


In [20]:
new2 = result[0].str.split(" - ", expand = True)

In [21]:
new2

,0,1
0,Whitehouse Station Sauce Company,Jalapeño
1,Jersey Barnfire,Black Garlic
2,Roofeeo Landis,Jah Mama Sauce
3,Ric Orlando's Best,Purple Haze Double Dose
4,Hell's Kitchen Hot Sauce,Pepper Pastry
5,Pickle Monster Hot Sauce,Spicy Boi Blend
6,Pickle Monster Hot Sauce,Pickle Monster
7,Swamp Dragon,Tequila Hot Sauce
8,Cepera,Molho de Pimenta Vermelha: Salsa de Aji Roja
9,Tacoma Sauce Co.,Nero


In [22]:
result1 = pd.concat([result, new2], axis=1, ignore_index=True)

In [23]:
result1

,0,1,2,3,4,5,6
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None,Whitehouse Station Sauce Company,Jalapeño
1,Jersey Barnfire - Black Garlic,None,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None,Jersey Barnfire,Black Garlic
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None,Roofeeo Landis,Jah Mama Sauce
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None,Ric Orlando's Best,Purple Haze Double Dose
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None,Hell's Kitchen Hot Sauce,Pepper Pastry
5,Pickle Monster Hot Sauce - Spicy Boi Blend,None,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","abanero, Jalapeño, Serrano",None,Pickle Monster Hot Sauce,Spicy Boi Blend
6,Pickle Monster Hot Sauce - Pickle Monster,None,"ickle Monster Hot Sauce, Philadelphia, PA, Uni...","Jalapeño, Serrano",None,Pickle Monster Hot Sauce,Pickle Monster
7,Swamp Dragon - Tequila Hot Sauce,None,"wamp Dragon, Baton Rouge, LA, United Stat",None,None,Swamp Dragon,Tequila Hot Sauce
8,Cepera - Molho de Pimenta Vermelha: Salsa de A...,None,"Cepera, Brazil\n",Jalapeño,None,Cepera,Molho de Pimenta Vermelha: Salsa de Aji Roja
9,Tacoma Sauce Co. - Nero,N/A,"Tacoma Sauce Co., Tacoma, United States\n ...",abanero\n \n,"3,000",Tacoma Sauce Co.,Nero
